## Data Processing

In [1]:
# Import the necessary libs
from keras.models import Model
from keras.layers import Input, Dense, Activation
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
sales = pd.read_csv('Data_With_critics.csv')
sales.head()

,Platform,Manufactur,Year,Publisher,NA,EU,JP,Other,Global,Critic_Score,Critic_Count,User_Score,User_Count,Developer
0,Wii,Nintendo,2006,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo
1,Wii,Nintendo,2008,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo
2,Wii,Nintendo,2009,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo
3,DS,Nintendo,2006,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo
4,Wii,Nintendo,2006,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo


In [5]:
X = sales.iloc[:, [0,1,2,3,9,10,11,12,13]]
y_na = sales['NA'].values
y_eu = sales['EU'].values
y_jp = sales['JP'].values
y_other = sales['Other'].values
X.head()

,Platform,Manufactur,Year,Publisher,Critic_Score,Critic_Count,User_Score,User_Count,Developer
0,Wii,Nintendo,2006,Nintendo,76.0,51.0,8.0,322.0,Nintendo
1,Wii,Nintendo,2008,Nintendo,82.0,73.0,8.3,709.0,Nintendo
2,Wii,Nintendo,2009,Nintendo,80.0,73.0,8.0,192.0,Nintendo
3,DS,Nintendo,2006,Nintendo,89.0,65.0,8.5,431.0,Nintendo
4,Wii,Nintendo,2006,Nintendo,58.0,41.0,6.6,129.0,Nintendo


In [6]:
# Onehotencode and check the nr of inputs for the first layer
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse=False), [0, 1, 2, 3, 8])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
X[0,1:].shape

(1625,)

### Splitting the dataset into the Training set and Test set

In [9]:
X_train, X_test, y_na_train, y_na_test, y_eu_train, y_eu_test , y_jp_train, y_jp_test , y_other_train, y_other_test  = train_test_split(X, y_na, y_eu, y_jp, y_other, test_size = 0.2, random_state = 0)

In [13]:
X_train.shape

(5514, 1626)

In [14]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the neural network

In [22]:
# Creating the input layer
# 1625 is the number of columns in the X + 1.
my_input = Input(shape=(1626, ))

In [23]:
# Creating first hidden layer
# (input shape + number of outputs) / 2 = (1625 + 4) / 2 = 814 rounded
# It's necessary to say which layer the hidden layer is connected to. In that case, the input layer
my_hidden_layer_1 = Dense(units=1169, activation='sigmoid')(my_input)

In [24]:
# Creating second hidden layer
my_hidden_layer_2 = Dense(units=814, activation='sigmoid')(my_hidden_layer_1)

In [25]:
# Create the output layers, one for each continent
# The activation function "linear" keeps the output values
my_output_layer_1 = Dense(units=1, activation='linear')(my_hidden_layer_2)
my_output_layer_2 = Dense(units=1, activation='linear')(my_hidden_layer_2)
my_output_layer_3 = Dense(units=1, activation='linear')(my_hidden_layer_2)
my_output_layer_4 = Dense(units=1, activation='linear')(my_hidden_layer_2)

In [26]:
# Create the model with the inputs and outputs
my_model = Model(inputs=my_input,
                outputs=[my_output_layer_1, my_output_layer_2, my_output_layer_3, my_output_layer_4])

In [27]:
#Compile the model
my_model.compile(optimizer='adam', loss='mse')

In [28]:
# Fit the model WARNING!!! This will take some time.
my_model.fit(X_train, [y_na_train, y_eu_train, y_jp_train, y_other_train],batch_size=128, epochs=1000)

Epoch 1/1000
44/44 [==============================] - 1s 16ms/step - loss: 2.6998 - dense_8_loss: 0.9758 - dense_9_loss: 0.5740 - dense_10_loss: 0.4521 - dense_11_loss: 0.6978
Epoch 2/1000
44/44 [==============================] - 1s 15ms/step - loss: 1.2170 - dense_8_loss: 0.6987 - dense_9_loss: 0.3644 - dense_10_loss: 0.0646 - dense_11_loss: 0.0893
Epoch 3/1000
44/44 [==============================] - 1s 15ms/step - loss: 1.1408 - dense_8_loss: 0.6709 - dense_9_loss: 0.3394 - dense_10_loss: 0.0559 - dense_11_loss: 0.0747
Epoch 4/1000
44/44 [==============================] - 1s 16ms/step - loss: 1.0915 - dense_8_loss: 0.6516 - dense_9_loss: 0.3289 - dense_10_loss: 0.0453 - dense_11_loss: 0.0657
Epoch 5/1000
44/44 [==============================] - 1s 15ms/step - loss: 1.0636 - dense_8_loss: 0.6318 - dense_9_loss: 0.3231 - dense_10_loss: 0.0445 - dense_11_loss: 0.0642: 0s - loss: 0.6971 - dense_8_loss: 0.3514 - dense_9_loss: 0.2224 - dense_10_loss: 0.0438 - dense_11_loss
Epoch 6/1000
44

44/44 [==============================] - 1s 16ms/step - loss: 0.4602 - dense_8_loss: 0.2540 - dense_9_loss: 0.1517 - dense_10_loss: 0.0244 - dense_11_loss: 0.0301
Epoch 46/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.4822 - dense_8_loss: 0.2618 - dense_9_loss: 0.1619 - dense_10_loss: 0.0264 - dense_11_loss: 0.0321
Epoch 47/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.4691 - dense_8_loss: 0.2602 - dense_9_loss: 0.1530 - dense_10_loss: 0.0235 - dense_11_loss: 0.0324
Epoch 48/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.4836 - dense_8_loss: 0.2720 - dense_9_loss: 0.1556 - dense_10_loss: 0.0240 - dense_11_loss: 0.0320
Epoch 49/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.4701 - dense_8_loss: 0.2573 - dense_9_loss: 0.1574 - dense_10_loss: 0.0247 - dense_11_loss: 0.0306
Epoch 50/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.4967 - dense_8_loss: 0.2809 - dense_9_loss: 0.1591 

44/44 [==============================] - 1s 16ms/step - loss: 0.3231 - dense_8_loss: 0.1836 - dense_9_loss: 0.1064 - dense_10_loss: 0.0194 - dense_11_loss: 0.0137
Epoch 91/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.3247 - dense_8_loss: 0.1864 - dense_9_loss: 0.1076 - dense_10_loss: 0.0171 - dense_11_loss: 0.0135
Epoch 92/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.3088 - dense_8_loss: 0.1755 - dense_9_loss: 0.1027 - dense_10_loss: 0.0167 - dense_11_loss: 0.0140
Epoch 93/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.2687 - dense_8_loss: 0.1548 - dense_9_loss: 0.0846 - dense_10_loss: 0.0172 - dense_11_loss: 0.0121
Epoch 94/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.2637 - dense_8_loss: 0.1512 - dense_9_loss: 0.0850 - dense_10_loss: 0.0155 - dense_11_loss: 0.0120
Epoch 95/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.2850 - dense_8_loss: 0.1609 - dense_9_loss: 0.0942 

44/44 [==============================] - 1s 16ms/step - loss: 0.1442 - dense_8_loss: 0.0805 - dense_9_loss: 0.0437 - dense_10_loss: 0.0126 - dense_11_loss: 0.0073
Epoch 135/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.1131 - dense_8_loss: 0.0642 - dense_9_loss: 0.0324 - dense_10_loss: 0.0100 - dense_11_loss: 0.0065
Epoch 136/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.1051 - dense_8_loss: 0.0609 - dense_9_loss: 0.0301 - dense_10_loss: 0.0081 - dense_11_loss: 0.0060
Epoch 137/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.1034 - dense_8_loss: 0.0581 - dense_9_loss: 0.0307 - dense_10_loss: 0.0084 - dense_11_loss: 0.0062
Epoch 138/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0814 - dense_8_loss: 0.0437 - dense_9_loss: 0.0237 - dense_10_loss: 0.0086 - dense_11_loss: 0.0053: 0s - loss: 0.0891 - dense_8_loss: 0.0460 - dense_9_loss: 0.0283 - dense_10_loss: 0.0087 - dense_11_loss: 0
Epoch 139/1000
44

44/44 [==============================] - 1s 15ms/step - loss: 0.0371 - dense_8_loss: 0.0195 - dense_9_loss: 0.0119 - dense_10_loss: 0.0034 - dense_11_loss: 0.0023
Epoch 179/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.0369 - dense_8_loss: 0.0205 - dense_9_loss: 0.0112 - dense_10_loss: 0.0028 - dense_11_loss: 0.0024
Epoch 180/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0326 - dense_8_loss: 0.0167 - dense_9_loss: 0.0107 - dense_10_loss: 0.0031 - dense_11_loss: 0.0022
Epoch 181/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0369 - dense_8_loss: 0.0174 - dense_9_loss: 0.0135 - dense_10_loss: 0.0037 - dense_11_loss: 0.0023
Epoch 182/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.0408 - dense_8_loss: 0.0213 - dense_9_loss: 0.0134 - dense_10_loss: 0.0033 - dense_11_loss: 0.0028
Epoch 183/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0394 - dense_8_loss: 0.0206 - dense_9_loss: 0.

...

44/44 [==============================] - 1s 16ms/step - loss: 0.0029 - dense_8_loss: 0.0020 - dense_9_loss: 3.2551e-04 - dense_10_loss: 3.2336e-04 - dense_11_loss: 2.6761e-04
Epoch 820/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0027 - dense_8_loss: 0.0017 - dense_9_loss: 4.0220e-04 - dense_10_loss: 3.1735e-04 - dense_11_loss: 2.3452e-04
Epoch 821/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.0030 - dense_8_loss: 0.0018 - dense_9_loss: 7.9370e-04 - dense_10_loss: 2.9433e-04 - dense_11_loss: 1.4465e-04 - loss: 0.0036 - dense_8_loss: 0.0023 - dense_9_loss: 9.1943e-04 - dense_10_loss: 2.6574e-04 - dense_11_loss: 1.54
Epoch 822/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0037 - dense_8_loss: 0.0017 - dense_9_loss: 6.8271e-04 - dense_10_loss: 0.0011 - dense_11_loss: 2.4059e-04
Epoch 823/1000
44/44 [==============================] - 1s 17ms/step - loss: 0.0093 - dense_8_loss: 0.0031 - dense_9_loss: 0.0044 - dense_10_los

44/44 [==============================] - 1s 16ms/step - loss: 0.0018 - dense_8_loss: 0.0011 - dense_9_loss: 3.1030e-04 - dense_10_loss: 2.4518e-04 - dense_11_loss: 1.1902e-04
Epoch 861/1000
44/44 [==============================] - 1s 17ms/step - loss: 0.0019 - dense_8_loss: 0.0013 - dense_9_loss: 3.6374e-04 - dense_10_loss: 2.1935e-04 - dense_11_loss: 8.6352e-05
Epoch 862/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0071 - dense_8_loss: 0.0042 - dense_9_loss: 0.0019 - dense_10_loss: 3.6595e-04 - dense_11_loss: 5.8933e-04
Epoch 863/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0040 - dense_8_loss: 0.0024 - dense_9_loss: 0.0011 - dense_10_loss: 3.1355e-04 - dense_11_loss: 1.9892e-04: 0s - loss: 0.0040 - dense_8_loss: 0.0024 - dense_9_loss: 0.0011 - dense_10_loss: 3.1355e-04 - dense_11_loss: 1.9892e-0
Epoch 864/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.0032 - dense_8_loss: 0.0021 - dense_9_loss: 6.7174e-04 - dense_1

44/44 [==============================] - 1s 16ms/step - loss: 0.0024 - dense_8_loss: 0.0011 - dense_9_loss: 2.5410e-04 - dense_10_loss: 6.0994e-04 - dense_11_loss: 4.4200e-04
Epoch 903/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.0024 - dense_8_loss: 0.0012 - dense_9_loss: 2.2434e-04 - dense_10_loss: 4.4948e-04 - dense_11_loss: 5.1559e-04
Epoch 904/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0021 - dense_8_loss: 0.0011 - dense_9_loss: 2.8342e-04 - dense_10_loss: 3.4336e-04 - dense_11_loss: 3.7427e-04
Epoch 905/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0018 - dense_8_loss: 0.0011 - dense_9_loss: 2.7687e-04 - dense_10_loss: 2.4523e-04 - dense_11_loss: 1.7074e-04
Epoch 906/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.0015 - dense_8_loss: 0.0011 - dense_9_loss: 1.9973e-04 - dense_10_loss: 1.9250e-04 - dense_11_loss: 7.1794e-05
Epoch 907/1000
44/44 [==============================] - 1s 17ms/s

44/44 [==============================] - 1s 16ms/step - loss: 0.0018 - dense_8_loss: 0.0012 - dense_9_loss: 2.3542e-04 - dense_10_loss: 2.3657e-04 - dense_11_loss: 9.5865e-05
Epoch 945/1000
44/44 [==============================] - 1s 17ms/step - loss: 0.0016 - dense_8_loss: 0.0011 - dense_9_loss: 1.7222e-04 - dense_10_loss: 2.0430e-04 - dense_11_loss: 9.1113e-05
Epoch 946/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.0015 - dense_8_loss: 0.0010 - dense_9_loss: 1.8352e-04 - dense_10_loss: 2.1911e-04 - dense_11_loss: 1.0477e-04
Epoch 947/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0017 - dense_8_loss: 0.0011 - dense_9_loss: 2.5055e-04 - dense_10_loss: 2.1014e-04 - dense_11_loss: 1.1352e-04
Epoch 948/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.0018 - dense_8_loss: 0.0012 - dense_9_loss: 2.2458e-04 - dense_10_loss: 2.2106e-04 - dense_11_loss: 1.1685e-04
Epoch 949/1000
44/44 [==============================] - 1s 16ms/s

44/44 [==============================] - 1s 15ms/step - loss: 0.0024 - dense_8_loss: 0.0013 - dense_9_loss: 3.9983e-04 - dense_10_loss: 2.8440e-04 - dense_11_loss: 3.8152e-04
Epoch 988/1000
44/44 [==============================] - 1s 15ms/step - loss: 0.0023 - dense_8_loss: 0.0012 - dense_9_loss: 3.1359e-04 - dense_10_loss: 2.8722e-04 - dense_11_loss: 4.4296e-04
Epoch 989/1000
44/44 [==============================] - 1s 17ms/step - loss: 0.0021 - dense_8_loss: 0.0011 - dense_9_loss: 4.2997e-04 - dense_10_loss: 3.2301e-04 - dense_11_loss: 2.2378e-04: 0s - loss: 0.0036 - dense_8_loss: 0.0020 - dense_9_loss: 7.1371e-04 - dense_10_loss: 4.4377e-04 - dense_11_loss: 4.
Epoch 990/1000
44/44 [==============================] - 1s 16ms/step - loss: 0.0022 - dense_8_loss: 0.0012 - dense_9_loss: 3.8880e-04 - dense_10_loss: 3.4609e-04 - dense_11_loss: 1.8699e-04
Epoch 991/1000
44/44 [==============================] - 1s 17ms/step - loss: 0.0021 - dense_8_loss: 0.0013 - dense_9_loss: 3.2504e-04 - de

### Predicting the results of the Test set

In [29]:
y_pred = my_model.predict(X_test)
y_pred

[array([[ 1.4801337 ],
        [ 0.508057  ],
        [-0.01057201],
        ...,
        [ 0.6490004 ],
        [ 0.05961001],
        [ 0.87649643]], dtype=float32),
 array([[ 1.14104   ],
        [ 0.21568803],
        [ 0.00511648],
        ...,
        [-0.18131872],
        [ 0.02372407],
        [ 0.8968009 ]], dtype=float32),
 array([[ 1.4490058 ],
        [-0.0082296 ],
        [-0.01472883],
        ...,
        [-0.09496944],
        [-0.04773409],
        [-0.00486263]], dtype=float32),
 array([[0.25555623],
        [0.09527914],
        [0.00053929],
        ...,
        [0.05638994],
        [0.1273032 ],
        [0.17814927]], dtype=float32)]

In [30]:
np.set_printoptions(precision=2)

In [31]:
y_pred = y_pred[0]
y_pred

array([[ 1.48],
       [ 0.51],
       [-0.01],
       ...,
       [ 0.65],
       [ 0.06],
       [ 0.88]], dtype=float32)

In [32]:
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_na_test.reshape(len(y_na_test),1)),1))

[[ 1.48  1.18]
 [ 0.51  0.24]
 [-0.01  0.01]
 ...
 [ 0.65  0.45]
 [ 0.06  0.19]
 [ 0.88  0.5 ]]


### Evaluate the test data

In [33]:
print("Evaluate on test data")
results = my_model.evaluate(X_test, [y_na_test, y_eu_test , y_jp_test , y_other_test], batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
11/11 [==============================] - 0s 4ms/step - loss: 1.0515 - dense_8_loss: 0.6180 - dense_9_loss: 0.2978 - dense_10_loss: 0.0762 - dense_11_loss: 0.0595
test loss, test acc: [1.051491618156433, 0.6180391311645508, 0.2977851331233978, 0.07620545476675034, 0.05946192517876625]


In [38]:
print("Accuracy North America: %.2f%%" % (results[1]*100))
print("Accuracy Europe: %.2f%%" % (results[2]*100))
print("Accuracy Japan: %.2f%%" % (results[3]*100))
print("Accuracy Other: %.2f%%" % (results[4]*100))
print("Total test Loss: %.2f%%" % (results[0]*100))

Accuracy North America: 61.80%
Accuracy Europe: 29.78%
Accuracy Japan: 7.62%
Accuracy Other: 5.95%
Total test Loss: 105.15%
